In [1]:
!pip install transformers
!pip install wandb
!pip install tensorboardX
!pip install ray
!pip install seqeval


     |████████████████████████████████| 3.4 MB 7.1 MB/s 
     |████████████████████████████████| 895 kB 76.5 MB/s 
     |████████████████████████████████| 3.3 MB 60.1 MB/s 
     |████████████████████████████████| 67 kB 7.6 MB/s 
     |████████████████████████████████| 596 kB 90.3 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
     |████████████████████████████████| 1.7 MB 8.9 MB/s 
     |████████████████████████████████| 142 kB 54.0 MB/s 
     |████████████████████████████████| 97 kB 8.7 MB/s 
     |████████████████████████████████| 180 kB 80.7 MB/s 
     |████████████████████████████████| 63 kB 2.4 MB/s 
  Created wheel for subprocess32: filename=subprocess32-3.5.4-py3-none-any.whl size=6502 sha256=b5bfd993cb5798060eb74d185d677cdace0b804654d5fb19f6a9544b9e48870b
  Stored in directory: /root/.cache/pip/wheels/50/ca/fa/8fca8d246e64f19488d07567547ddec8eb084e8c0d7a59226a
  Created w

In [2]:
!git clone https://github.com/cjGO/yaamformer.git

Cloning into 'yaamformer'...
remote: Enumerating objects: 70, done.
remote: Counting objects: 100% (70/70), done.
remote: Compressing objects: 100% (53/53), done.
remote: Total 70 (delta 24), reused 52 (delta 11), pack-reused 0
Unpacking objects: 100% (70/70), done.


In [ ]:
#!wandb login

wandb: Currently logged in as: glector (use `wandb login --relogin` to force relogin)


In [24]:
from ast import literal_eval
from sklearn.model_selection import train_test_split
from transformers import BertForMaskedLM, BertTokenizer, pipeline
from yaamformer.utils import *

In [26]:
#PARAMETERS

args = {'MAX_LENGTH':1024, #max length sequence to keep in training dataset.
                           #pads shorter sequence to this length
        'DATA_SEED':42, #random state for splitting datasets w/ sklearn
        
        }

In [6]:
import pandas as pd
import numpy as np
import re
import torch

Seqs,Labels = get_dataset('./yaamformer/data/YAAM.txt')

In [8]:
sum_seq = 0
for i in Seqs:
  sum_seq += len(i)

  sum_rez = 0
for i in Labels:
  sum_rez += sum(i)

print(f'Approximately {round(sum_rez/sum_seq,3)*100}% of amino acids are modified.')

Approximately 2.1% of amino acids are modified.


In [17]:
yaam = pd.DataFrame({'seq':Seqs,
                     'label':Labels})

yaam['len'] = yaam['seq'].map(len) #collect sequence lengths to filter
yaam = yaam[yaam['len'] < args['MAX_LENGTH']] #filter based on argument max_length
print(f'There are {len(yaam)} sequences available for model training')

There are 4385 sequences available for model training


In [22]:
#pad the labels to match the max_length (all sequences are max_length for training)
yaam['label'] = yaam['label'].apply(pad_labels,max_length=1024)

In [27]:
train, test = train_test_split(yaam, test_size=0.2, random_state=args['DATA_SEED'])
test = test.sample(frac=1)
train = train.reset_index()
test = test.reset_index()

In [ ]:
#model = BertForTokenClassification.from_pretrained("Rostlab/prot_bert",)

In [28]:
train_encodings = [tokenize_protein(x) for x in train['seq']]
test_encodings = [tokenize_protein(x) for x in test['seq']]

tokenizer = BertTokenizer.from_pretrained("Rostlab/prot_bert", do_lower_case=False, max_length=args['MAX_LENGTH'])
train_encodings = tokenizer(train_encodings, padding='max_length', max_length=args['MAX_LENGTH'], truncation=True)
test_encodings = tokenizer(test_encodings, padding='max_length', max_length=args['MAX_LENGTH'], truncation=True)#padding=True

train_labels = train['label']
test_labels = test['label']

train_dataset = NERprotein(train_encodings, train_labels)
test_dataset = NERprotein(test_encodings, test_labels)

NameError: ignored

In [ ]:
for i in train_encodings:
  print(i)

input_ids
token_type_ids
attention_mask


In [ ]:
from transformers import BertModel, BertTokenizer, BertForTokenClassification
import re
from transformers import DataCollatorForTokenClassification
from transformers import TrainingArguments, Trainer
from typing import Any 
from transformers import Trainer

## HYPER PARAMETER TUNING



In [ ]:

from dataclasses import dataclass, field

@dataclass
class CustomTrainingArguments(TrainingArguments):

    weight_picks_0: float = field(
        default=None, metadata={"help": "Batch size per GPU/TPU core/CPU for evaluation."}
    )

    weight_picks_1: float = field(
        default=None, metadata={"help": "Batch size per GPU/TPU core/CPU for evaluation."}
    )


class CustomTrainer(Trainer):

    def __init__(self, *args, **kwargs):
        super(CustomTrainer, self).__init__(*args, **kwargs)
        

    def _hp_search_setup(self, trial: Any):
        try:
            trial.pop("wandb", None)
        except AttributeError:
            pass
        super(CustomTrainer, self)._hp_search_setup(trial)

    def compute_loss(self, model, inputs, return_outputs=False):
        #print('hello!')
        labels = inputs.get("labels")
        outputs = model(**inputs)
        logits = outputs.get("logits")

#        weights = torch.tensor([1.,49.])
        weight_picks = torch.tensor([training_args.weight_picks_0,training_args.weight_picks_1])
        #print(weight_picks)
        loss_fct = torch.nn.CrossEntropyLoss(weight_picks.to('cuda'))
        loss = loss_fct(logits.view(-1, 2), labels.view(-1))

        # loss = loss_fct(
        #     logits.view(-1, self.model.config.num_labels), labels.float().view(-1, self.model.config.num_labels)
        # )
        return (loss, outputs) if return_outputs else loss

In [ ]:
!wandb login
def model_init():
    model = BertForTokenClassification.from_pretrained("Rostlab/prot_bert",)
    return model

def f1(predictions):
  class_probs = predictions[0]
  true_labels = predictions[1]

  f1s = []
  for i in range(len(class_probs)):
    cp = class_probs[i]
    tl = true_labels[i]
    pred_labels = [np.argmax(x) for x in class_probs[0]]
    f1s.append(f1_score(tl,pred_labels))
  return {'f1':np.mean(f1s)}


wandb: Currently logged in as: glector (use `wandb login --relogin` to force relogin)


In [ ]:
from ray import tune
from transformers.integrations import WandbCallback
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

data_collator = DataCollatorForTokenClassification(tokenizer=tokenizer,padding=True,)



for batch_size in [1]:
  for weight_picks_1 in [2.]:
    for label_smoothing_factor in [.0]:
        

      training_args = CustomTrainingArguments(
        output_dir='./',          # output directory to where save model checkpoint
        evaluation_strategy="steps",    # evaluate each `logging_steps` steps
        overwrite_output_dir=True,      
        num_train_epochs=1,            # number of training epochs, feel free to tweak
        per_device_train_batch_size=batch_size, # the training batch size, put it as high as your GPU memory fits
        gradient_accumulation_steps=1,  # accumulating the gradients before updating the weights
        per_device_eval_batch_size=2,  # evaluation batch size
        logging_steps=50,             # evaluate, log and save model checkpoints every 1000 step
        save_steps=200,
        gradient_checkpointing = True,
        report_to='wandb',
        weight_picks_0 = 1.,
        weight_picks_1 = weight_picks_1,
        label_smoothing_factor = label_smoothing_factor,
        run_name=(f'pick1_{weight_picks_1}_smooth{label_smoothing_factor}_bs{batch_size}')



      #load_best_model_at_end=False,  # whether to load the best model (in terms of loss) at the end of training
      # save_total_limit=3,
      #  weight_picks_0 = 3.,
      #  weight_picks_1= 6.,
      #label_smoothing_factor=.05
                  # whether you don't have much space so you let only 3 model weights saved in the disk
    )

    trainer = CustomTrainer(
        model_init=model_init,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=test_dataset,
        compute_metrics=f1,
        tokenizer=tokenizer,
        data_collator=data_collator)
    
    trainer.train()


loading configuration file https://huggingface.co/Rostlab/prot_bert/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/26014e822b9642c94410e0829f3080e5aa601146ffdff9eaafda9ca18a3394e7.baf557855a8618d0ddfb6c23bfd135bfc38ccf8c3fb099b8df45eb110ccf05e9
Model config BertConfig {
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.0,
  "classifier_dropout": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.0,
  "hidden_size": 1024,
  "initializer_range": 0.02,
  "intermediate_size": 4096,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 40000,
  "model_type": "bert",
  "num_attention_heads": 16,
  "num_hidden_layers": 30,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.15.0",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30
}

loading weights file https://huggingface.co/Rostlab/prot_bert/resolve/main/pytorch_model.bin from cache at /root/.cache/huggingface/transfo

/usr/local/lib/python3.7/dist-packages/transformers/data/data_collator.py:317: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  sequence_length = torch.tensor(batch["input_ids"]).shape[1]
/usr/local/lib/python3.7/dist-packages/transformers/data/data_collator.py:328: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  batch = {k: torch.tensor(v, dtype=torch.int64) for k, v in batch.items()}


Step,Training Loss,Validation Loss,F1
50,0.140300,0.096186,0.000000
100,0.103500,0.096006,0.000000
150,0.108500,0.090987,0.000000
200,0.080400,0.085511,0.000000
250,0.096700,0.086808,0.000000
300,0.075400,0.081252,0.000000
350,0.079700,0.073358,0.000000
400,0.074200,0.069252,0.000000
450,0.077500,0.067968,0.000000
500,0.075400,0.072067,0.000000


***** Running Evaluation *****
  Num examples = 877
  Batch size = 2
/usr/local/lib/python3.7/dist-packages/transformers/data/data_collator.py:317: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  sequence_length = torch.tensor(batch["input_ids"]).shape[1]
/usr/local/lib/python3.7/dist-packages/transformers/data/data_collator.py:328: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  batch = {k: torch.tensor(v, dtype=torch.int64) for k, v in batch.items()}
***** Running Evaluation *****
  Num examples = 877
  Batch size = 2
/usr/local/lib/python3.7/dist-packages/transformers/data/data_collator.py:317: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().det

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
def compute_metrics(p):
    pred, labels = p
    
    print(f'pred: {pred} \n label:{labels}')
    pred = np.argmax(pred, axis=1)
    rounded_labels=np.argmax(labels, axis=1)
    accuracy = accuracy_score(y_true=labels, y_pred=pred)
    recall = recall_score(y_true=labels, y_pred=pred)
    precision = precision_score(y_true=labels, y_pred=pred)
    f1 = f1_score(y_true=labels, y_pred=pred)

    return {"accuracy": accuracy, "precision": precision, "recall": recall, "f1": f1}

In [ ]:
f1.name = 'f1'

In [ ]:
f1.metric = 'f1'

In [ ]:
#model = BertForTokenClassification.from_pretrained("Rostlab/prot_bert",)

import numpy as np
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

def accuracy(predictions):
  class_probs = predictions[0]
  true_labels = predictions[1]

  f1s = []
  for i in range(len(class_probs)):
    cp = class_probs[i]
    tl = true_labels[i]
    pred_labels = [np.argmax(x) for x in class_probs[0]]
    f1s.append(f1_score(tl,pred_labels))
  return {'accuracy':np.mean(f1s)}

# trainer = CustomTrainer(
    
#     model=model,
#     args=training_args,
#     data_collator=data_collator,
#     train_dataset=train_dataset,
#     eval_dataset=test_dataset,
#     compute_metrics  = fix_metric,
# )



# trainer.pop_callback(WandbCallback)

# trainer.train()

In [ ]:
#from sklearn.metrics import f1_score
!pip install datasets

     |████████████████████████████████| 306 kB 7.1 MB/s 
     |████████████████████████████████| 243 kB 72.8 MB/s 
     |████████████████████████████████| 1.1 MB 77.1 MB/s 
     |████████████████████████████████| 133 kB 96.9 MB/s 
     |████████████████████████████████| 192 kB 89.2 MB/s 
     |████████████████████████████████| 271 kB 54.7 MB/s 
     |████████████████████████████████| 160 kB 97.0 MB/s 


In [ ]:
import datasets

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

In [ ]:
# from ray import tune
# from transformers.integrations import WandbCallback

def model_init():
    model = BertForTokenClassification.from_pretrained("Rostlab/prot_bert",)
    return model

# def hp_space_fn(*args, **kwargs):
#     config = {
#                 "learning_rate": tune.choice([2e-5, 3e-5]),
#                 "num_train_epochs": tune.choice([.005]),
#                  'label_smoothing_factor': tune.choice([0.01,0.03,.07,.10]),
#                  'weight_picks_0' : tune.choice([0.01,0.03]),
#                  'weight_picks_1' : tune.choice([0.05,.07,.10,.2,.5]),
#     }
#     wandb_config = {
#             "wandb": {
#                     "project": os.environ.get(
#                         "WANDB_PROJECT",
#                         "wandb_project"),
#                     "api_key": os.environ.get("API_KEY"),
#                     "log_config": True
#                     }
#     }

#     config.update(wandb_config)
#     return config

trainer = CustomTrainer(
        model_init=model_init,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=test_dataset,
        compute_metrics=f1,
        tokenizer=tokenizer,
        data_collator=data_collator)

# #trainer.pop_callback(WandbCallback)


loading configuration file https://huggingface.co/Rostlab/prot_bert/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/26014e822b9642c94410e0829f3080e5aa601146ffdff9eaafda9ca18a3394e7.baf557855a8618d0ddfb6c23bfd135bfc38ccf8c3fb099b8df45eb110ccf05e9
Model config BertConfig {
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.0,
  "classifier_dropout": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.0,
  "hidden_size": 1024,
  "initializer_range": 0.02,
  "intermediate_size": 4096,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 40000,
  "model_type": "bert",
  "num_attention_heads": 16,
  "num_hidden_layers": 30,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.15.0",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30
}

loading weights file https://huggingface.co/Rostlab/prot_bert/resolve/main/pytorch_model.bin from cache at /root/.cache/huggingface/transfo

In [ ]:
DEFAULT_LOGGERS

NameError: ignored

In [ ]:
# from ray.tune.logger import DEFAULT_LOGGERS
# from ray.tune.integration.wandb import WandbLogger
# from ray.tune.schedulers import PopulationBasedTraining
# import os



# #Trial returned a result which did not include the specified metric(s) `f1`
# best_run = trainer.hyperparameter_search(
#             direction="minimize",
#             backend="ray",
#             hp_space=hp_space_fn,
#             loggers=(DEFAULT_LOGGERS + WandbLogger,),
#             #compute_metrics = accuracy,
#             n_trials=1#+ (WandbLogger, ),
#     )

# #logger.info(json.dumps(best_run.hyperparameters, indent=4))

In [ ]:
# %env WANDB_PROJECT=test
# from ray.tune.logger import DEFAULT_LOGGERS


# training_args = CustomTrainingArguments(
#     output_dir='./',          # output directory to where save model checkpoint
#     evaluation_strategy="steps",    # evaluate each `logging_steps` steps
#     overwrite_output_dir=True,      
#     #num_train_epochs=10,            # number of training epochs, feel free to tweak
#     per_device_train_batch_size=1, # the training batch size, put it as high as your GPU memory fits
#     gradient_accumulation_steps=1,  # accumulating the gradients before updating the weights
#     per_device_eval_batch_size=4,  # evaluation batch size
#     logging_steps=10,             # evaluate, log and save model checkpoints every 1000 step
#     save_steps=50,
#     gradient_checkpointing = True,
#     #report_to='wandb',
# #

#      #load_best_model_at_end=False,  # whether to load the best model (in terms of loss) at the end of training
#      #save_total_limit=3,
#     #  weight_picks_0 = 3.,
#     #  weight_picks_1= 6.,
#      #label_smoothing_factor=.05
#                 # whether you don't have much space so you let only 3 model weights saved in the disk
# )




# from ray import tune
# from transformers.integrations import WandbCallback

# def model_init():
#     model = BertForTokenClassification.from_pretrained("Rostlab/prot_bert",)
#     return model

# def hp_space_fn(*args, **kwargs):
#     config = {
#                 "learning_rate": tune.choice([2e-5, 3e-5]),
#                 "num_train_epochs": tune.choice([.005]),
#                 'label_smoothing_factor': tune.choice([0.01,0.03,.07,.10]),
#                 'weight_picks_0' : tune.choice([0.01,0.03]),
#                 'weight_picks_1' : tune.choice([0.05,.07,.10,.2,.5]),
#     }
#     wandb_config = {
#             "wandb": {
#                     "project": os.environ.get(
#                         "WANDB_PROJECT",
#                         "wandb_project"),
#                     "api_key": os.environ.get("API_KEY"),
#                     "log_config": False
#                     }
#     }

#     config.update(wandb_config)
#     return config




# def hp_space_fn(*args, **kwargs):
#     config = {
#                 "learning_rate": tune.choice([2e-5, 3e-5]),
#                 "num_train_epochs": tune.choice([.005]),
#                 'label_smoothing_factor': tune.choice([0.01,0.03,.07,.10]),
#                 'weight_picks_0' : tune.choice([0.01,0.03]),
#                 'weight_picks_1' : tune.choice([0.05,.07,.10,.2,.5]),
#     }
#     wandb_config = {
#             "wandb": {
#                     "project": os.environ.get(
#                         "WANDB_PROJECT",
#                         "wandb_project"),
#                     "api_key": os.environ.get("API_KEY"),
#                     "log_config": True
#                     }
#     }
#     config.update(wandb_config)
#     return config




# trainer = CustomTrainer(
#         model_init=model_init,
#         args=training_args,
#         train_dataset=train_dataset,
#         eval_dataset=test_dataset,
#         compute_metrics=f1,
#         tokenizer=tokenizer,
#         data_collator=data_collator)

# trainer.train()
# #trainer.pop_callback(WandbCallback)

# from ray.tune.logger import DEFAULT_LOGGERS
# from ray.tune.integration.wandb import WandbLogger
# from ray.tune.schedulers import PopulationBasedTraining
# import os


# #Trial returned a result which did not include the specified metric(s) `f1`
# best_run = trainer.hyperparameter_search(
#             direction="minimize",
#             backend="ray",
#             hp_space=hp_space_fn,
#             loggers=DEFAULT_LOGGERS + (WandbLogger,),
#             #compute_metrics = accuracy,
#             n_trials=2#+ (WandbLogger, ),
#     )

# #logger.info(json.dumps(best_run.hyperparameters, indent=4))




In [ ]:
%env WANDB_PROJECT=sing
from ray.tune.logger import DEFAULT_LOGGERS


training_args = CustomTrainingArguments(
    output_dir='./',          # output directory to where save model checkpoint
    evaluation_strategy="steps",    # evaluate each `logging_steps` steps
    overwrite_output_dir=True,      
    #num_train_epochs=10,            # number of training epochs, feel free to tweak
    per_device_train_batch_size=1, # the training batch size, put it as high as your GPU memory fits
    gradient_accumulation_steps=1,  # accumulating the gradients before updating the weights
    per_device_eval_batch_size=4,  # evaluation batch size
    logging_steps=10,             # evaluate, log and save model checkpoints every 1000 step
    save_steps=50,
    gradient_checkpointing = True,
    weight_picks_0 = 1.,
    weight_picks_1=7.,
    #report_to='wandb',
#

     #load_best_model_at_end=False,  # whether to load the best model (in terms of loss) at the end of training
     #save_total_limit=3,
    #  weight_picks_0 = 3.,
    #  weight_picks_1= 6.,
     #label_smoothing_factor=.05
                # whether you don't have much space so you let only 3 model weights saved in the disk
)




from ray import tune
from transformers.integrations import WandbCallback

def model_init():
    model = BertForTokenClassification.from_pretrained("Rostlab/prot_bert",)
    return model

def hp_space_fn(*args, **kwargs):
    config = {
                "learning_rate": tune.choice([2e-5, 3e-5]),
                "num_train_epochs": tune.choice([.005]),
                'label_smoothing_factor': tune.choice([0.01,0.03,.07,.10]),
                'weight_picks_0' : tune.choice([0.01,0.03]),
                'weight_picks_1' : tune.choice([0.05,.07,.10,.2,.5]),
    }
    wandb_config = {
            "wandb": {
                    "project": os.environ.get(
                        "WANDB_PROJECT",
                        "wandb_project"),
                    "api_key": os.environ.get("API_KEY"),
                    "log_config": True
                    }
    }

    config.update(wandb_config)
    return config




def hp_space_fn(*args, **kwargs):
    config = {
                "learning_rate": tune.choice([2e-5, 3e-5]),
                "num_train_epochs": tune.choice([.005]),
                'label_smoothing_factor': tune.choice([0.01,0.03,.07,.10]),
                'weight_picks_0' : tune.choice([0.01,0.03]),
                'weight_picks_1' : tune.choice([0.05,.07,.10,.2,.5]),
    }
    wandb_config = {
            "wandb": {
                    "project": os.environ.get(
                        "WANDB_PROJECT",
                        "wandb_project"),
                    "api_key": os.environ.get("API_KEY"),
                    "log_config": True
                    }
    }
    config.update(wandb_config)
    return config




trainer = CustomTrainer(
        model_init=model_init,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=test_dataset,
        compute_metrics=f1,
        tokenizer=tokenizer,
        data_collator=data_collator)

trainer.train()

In [ ]:
def hp_space_fn(*args, **kwargs):
    config = {
                "learning_rate": tune.choice([2e-5, 3e-5]),
                "num_train_epochs": tune.choice([.005]),
                 'label_smoothing_factor': tune.choice([0.01,0.03,.07,.10]),
                 'weight_picks_0' : tune.choice([0.01,0.03]),
                 'weight_picks_1' : tune.choice([0.05,.07,.10,.2,.5]),
    }
    wandb_config = {
            "wandb": {
                    "project": os.environ.get(
                        "WANDB_PROJECT",
                        "wandb_project"),
                    "api_key": os.environ.get("API_KEY"),
                    "log_config": True
                    }
    }

    config.update(wandb_config)
    return config

x = hp_space_fn()

In [ ]:
x

In [ ]:
!wandb login

In [ ]:
mylog = WandbLogger
mylog.update_config(mylog, config=hp_space_fn())

In [ ]:
mylog

In [ ]:
hp_space_fn()